In [ ]:
# Set environment
arcpy.CheckOutExtension("Spatial")
arcpy.env.workspace = "environment_path.gdb"
arcpy.env.overwriteOutput = True

import re

In [ ]:
# Delete last files to avoid overwriting
for file in arcpy.ListFeatureClasses('Watershed_poly_*'):
    arcpy.management.Delete(file)
    print(f"Removing: {file}")

In [ ]:
# Load fluxes rasters --> made with ArcGIs Pro user interface
flow_acc = arcpy.Raster("flow_accumulation_raster.tiff")
flow_dir = arcpy.Raster("flow_direction_raster.tif")
print("Fluxes rasters loaded.")

In [ ]:
i = 0
total = arcpy.GetCount_management("sampled_points")[0]

# Iterate hydrology function through sampled points
for row in arcpy.da.SearchCursor("sampled_points", ["ID", "SHAPE@"]):
    Value = row[0]
    Value = Value.replace("-","_") # To allow using ID name in filename
    geometry = row[1]
    print(f"Processing: ID Terrain: {Value}, Geometry: {geometry}")
    
    success = False
    
    # Try block used to handle errors
    # In case of error in the try block, 3 attempts are done to avoid "random" errors
    for attempt in range(3):
        try:
            Snap_pour_raster = arcpy.sa.SnapPourPoint(geometry, flow_acc, 0)

            Watersh = arcpy.sa.Watershed(flow_dir, Snap_pour_raster)
            Watershed_poly = f"Watershed_poly_{Value}"

            print("Point polygon saved at: ", Watershed_poly)

            arcpy.conversion.RasterToPolygon(in_raster = Watersh, out_polygon_features = Watershed_poly)
            success = True
            break
        except Exception as e:
            print(f"Attempt {attempt + 1}/3 failed for ID {Value}. Error: {e}")
    
    if not success:
        print(f"##### Failed to process ID {Value} after 3 attempts #####\n")
        
    arcpy.management.ClearWorkspaceCache()
    
    i = i+1
    print(f"{i}/{total}")
    
print("Point watershed processing finished.")

In [ ]:
# Merging all resulting files ans saving output

arcpy.management.Delete("merge_watershed_final.shp")

watershed_poly_features = []
for fc in arcpy.ListFeatureClasses('Watershed_poly_*'):
        watershed_poly_features.append(fc)

arcpy.management.Merge(watershed_poly_features, "merge_watershed_final", "","ADD_SOURCE_INFO")
arcpy.conversion.ExportFeatures("merge_watershed_final", "merge_watershed_final.shp")

print("Algorithm successfully finished")

arcpy.CheckInExtension("Spatial")